In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from preprocess import listFiles


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# file_path = 'datasets/json/box_plant1/box_plant1_frame300_400.json'
# # Open and read the JSON file
# with open(file_path, 'r') as file:
#     data = json.load(file)

# # Now 'data' is a dictionary
# num_key = len(data.keys())
# if num_key > 0:
#     print(f"Data loaded, number of frames = {num_key}")
# frame = 350
# points = np.array(data[str(frame)])

Data loaded, number of frames = 100


In [2]:
def quickVisualize(points):
    """ Visualize pointcloud from numpy array"""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.visualization.draw_geometries([pcd])
    return

In [3]:
def loadData(path):
    # List all files in the specified path, ignoring directories
    df = pd.read_csv(path)
    points_xyz = df.to_numpy()
    return points_xyz[:,8:11]

In [4]:
### for handling entier point cloud
def shortPassFilter(points, threshold = 10):
    """ Filter points that are further than a distance away"""
    distance = (points[:,0]**2 + points[:,1]**2 + points[:,2]**2)**0.5
    mask = distance < threshold
    return points[mask]

# prepare path
directory = r'datasets/json/box_plant1/'
files = listFiles(directory)
files.sort()

# prepare frame 0 as reference
path = directory + files[0]
with open(path, 'r') as file:
    data = json.load(file)
pts_frame_0 = np.array(data[str(0)])
pts_frame_60 = np.array(data[str(60)])
pts_frame_0 = shortPassFilter(pts_frame_0, 7)
pts_frame_60 = shortPassFilter(pts_frame_60, 7)


In [13]:
### For handling manual point clouds
path1 = r'datasets/box_plant1_manual_frame0.csv'
path2 = r'datasets/box_plant1_manual_frame60.csv'
pts_frame_0 = loadData(path1)
pts_frame_60 = loadData(path2)

In [5]:
# Create Open3D PointCloud objects
source = o3d.geometry.PointCloud()
source.points = o3d.utility.Vector3dVector(pts_frame_0)

target= o3d.geometry.PointCloud()
target.points = o3d.utility.Vector3dVector(pts_frame_60)

In [6]:
### Global Registration
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_plotly([source_temp, target_temp])


def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh


def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    print(result.transformation)
    return result



In [7]:
voxel_size=0.05
source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)

:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


In [ ]:
result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

In [ ]:
### Applying ICP for local registration
trans_init = np.asarray(result_ransac.transformation)
threshold=0.02
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

In [4]:
from register import fullRegistration, loadData

In [20]:
### Choose directory here
directory = r'datasets/csv/box_plant1_manual/'
files = listFiles(directory)
files.sort()
pcds = []
for file in files:
    path = directory + file
    pcds.append(loadData(path))

pose_graph = fullRegistration(pcds)
breakpoint()

Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.reg

In [ ]:
# Apply the transformations to each point cloud
aligned_pcds = []
for i, node in enumerate(pose_graph.nodes):
    # Get the transformation matrix
    transformation = node.pose
    print(transformation)
    pcd_temp = copy.deepcopy(pcds[i])
    pcd_temp.transform(transformation)
    # pcd_transformed = pcds[i].transform(transformation)
    # pcd_transformed = pcds[i].transform(np.eye(4))
    if i == 0 or i == 1:
        aligned_pcds.append(pcd_temp)

combined_pcd = o3d.geometry.PointCloud()
for pcd in aligned_pcds:
    combined_pcd += pcd

combined_pcd_downsampled = combined_pcd.voxel_down_sample(voxel_size=0.02)
o3d.visualization.draw_geometries([combined_pcd_downsampled])

# Save the combined point cloud
# o3d.io.write_point_cloud("combined_point_cloud.ply", combined_pcd_downsampled)